In [1]:
import pandas as pd 
import geopandas
import numpy as np
import yellowcab.cleaning

In [2]:
cab_df = pd.read_parquet("../data/trip_data/01.parquet")
cab_df

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.20,1.0,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.60,1.0,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.80,1.0,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.00,1.0,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339562,2020-01-31 23:38:07,2020-01-31 23:52:21,1.0,2.10,1.0,163,246,1.0,11.0,0.5,0.5,2.96,0.0,0.3,17.76,2.5
6339563,2020-01-31 23:00:18,2020-01-31 23:19:18,1.0,2.13,1.0,164,79,1.0,13.0,0.5,0.5,3.36,0.0,0.3,20.16,2.5
6339564,2020-01-31 23:24:22,2020-01-31 23:40:39,1.0,2.55,1.0,79,68,1.0,12.5,0.5,0.5,3.26,0.0,0.3,19.56,2.5
6339565,2020-01-31 23:44:22,2020-01-31 23:54:00,1.0,1.61,1.0,100,142,2.0,8.5,0.5,0.5,0.00,0.0,0.3,12.30,2.5


In [3]:
cab_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6339567 entries, 0 to 6339566
Data columns (total 16 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   tpep_pickup_datetime   datetime64[ns]
 1   tpep_dropoff_datetime  datetime64[ns]
 2   passenger_count        float64       
 3   trip_distance          float64       
 4   RatecodeID             float64       
 5   PULocationID           int64         
 6   DOLocationID           int64         
 7   payment_type           float64       
 8   fare_amount            float64       
 9   extra                  float64       
 10  mta_tax                float64       
 11  tip_amount             float64       
 12  tolls_amount           float64       
 13  improvement_surcharge  float64       
 14  total_amount           float64       
 15  congestion_surcharge   float64       
dtypes: datetime64[ns](2), float64(12), int64(2)
memory usage: 822.2 MB


In [4]:
taxi_zone_df = pd.read_csv("../data/taxi_zones/taxi_zones.csv")
taxi_zone_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LocationID    265 non-null    int64 
 1   Borough       265 non-null    object
 2   Zone          264 non-null    object
 3   service_zone  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


In [5]:
geo_df = geopandas.read_file("../data/taxi_zones/taxi_zones.geojson") 

joined_gdf = geo_df.merge(taxi_zone_df,how='left', on="LocationID")

joined_gdf.set_index("OBJECTID")

,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,Borough,Zone,service_zone
OBJECTID,,,,,,,,,
1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19...",EWR,Newark Airport,EWR
2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343...",Queens,Jamaica Bay,Boro Zone
3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.770 256767.698, 1026495.593 ...",Bronx,Allerton/Pelham Gardens,Boro Zone
4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20...",Manhattan,Alphabet City,Yellow Zone
5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.310 144283.336, 936046.565 14...",Staten Island,Arden Heights,Boro Zone
...,...,...,...,...,...,...,...,...,...
259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((1025414.782 270986.139, 1025138.624 ...",Bronx,Woodlawn/Wakefield,Boro Zone
260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((1011466.966 216463.005, 1011545.889 ...",Queens,Woodside,Boro Zone
261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((980555.204 196138.486, 980570.792 19...",Manhattan,World Trade Center,Yellow Zone


In [6]:
"""cab_df = yellowcab.cleaning.get_date_components(cab_df)
joined_gdf = yellowcab.cleaning.get_position(joined_gdf)
cab_df = yellowcab.cleaning.to_int(cab_df)
cab_df = yellowcab.cleaning.get_duration(cab_df)
cab_df = yellowcab.cleaning.is_weekend(cab_df)
cab_df = yellowcab.cleaning.remove_outliers(cab_df)
cab_df"""

'cab_df = yellowcab.cleaning.get_date_components(cab_df)\njoined_gdf = yellowcab.cleaning.get_position(joined_gdf)\ncab_df = yellowcab.cleaning.to_int(cab_df)\ncab_df = yellowcab.cleaning.get_duration(cab_df)\ncab_df = yellowcab.cleaning.is_weekend(cab_df)\ncab_df = yellowcab.cleaning.remove_outliers(cab_df)\ncab_df'

In [7]:
"""relevant_columns = ['passenger_count', 'trip_distance', 'RatecodeID',
                    'PULocationID', 'DOLocationID', 'payment_type',
                    'fare_amount', 'extra', 'mta_tax', 'tip_amount',
                    'tolls_amount', 'improvement_surcharge', 'total_amount',
                    'congestion_surcharge']"""

"relevant_columns = ['passenger_count', 'trip_distance', 'RatecodeID',\n                    'PULocationID', 'DOLocationID', 'payment_type',\n                    'fare_amount', 'extra', 'mta_tax', 'tip_amount',\n                    'tolls_amount', 'improvement_surcharge', 'total_amount',\n                    'congestion_surcharge']"

In [8]:
"""
for c in relevant_columns:
    print("---- %s ---" % c)
    print(cab_df[c].value_counts())"""

'\nfor c in relevant_columns:\n    print("---- %s ---" % c)\n    print(cab_df[c].value_counts())'

In [9]:
"""
cab_df[cab_df["fare_amount"] == 65.25].drop(["start_month", "start_day", "start_hour", "end_month", "end_day", "end_hour"], axis=1)
"""

'\ncab_df[cab_df["fare_amount"] == 65.25].drop(["start_month", "start_day", "start_hour", "end_month", "end_day", "end_hour"], axis=1)\n'

In [10]:
"""
cab_df[cab_df["DOLocationID"] == 1]"""

'\ncab_df[cab_df["DOLocationID"] == 1]'

In [11]:
%timeit
yellowcab.cleaning.clean_dataset(cab_df, "fare_amount")

Shape with outliers: (4437696, 21) (4437696,)
MAE with outliers: 0.301
Shape isolation forest: (3816108, 21) (3816108,)
MAE isolation forest: 0.308


,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,...,total_amount,congestion_surcharge,start_month,start_day,start_hour,end_month,end_day,end_hour,weekend,trip_duration_minutes
0,1,1.20,1.0,238,239,1.0,6.0,3.0,0.5,1.47,...,11.27,2.5,1,1,0,1,1,0,False,4.800000
1,1,1.20,1.0,239,238,1.0,7.0,3.0,0.5,1.50,...,12.30,2.5,1,1,0,1,1,0,False,7.416667
2,1,0.60,1.0,238,238,1.0,6.0,3.0,0.5,1.00,...,10.80,2.5,1,1,0,1,1,0,False,6.183333
3,1,0.80,1.0,238,151,1.0,5.5,0.5,0.5,1.36,...,8.16,0.0,1,1,0,1,1,1,False,4.850000
4,1,0.00,1.0,193,193,2.0,3.5,0.5,0.5,0.00,...,4.80,0.0,1,1,0,1,1,0,False,2.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339562,1,2.10,1.0,163,246,1.0,11.0,0.5,0.5,2.96,...,17.76,2.5,1,31,23,1,31,23,False,14.233333
6339563,1,2.13,1.0,164,79,1.0,13.0,0.5,0.5,3.36,...,20.16,2.5,1,31,23,1,31,23,False,19.000000
6339564,1,2.55,1.0,79,68,1.0,12.5,0.5,0.5,3.26,...,19.56,2.5,1,31,23,1,31,23,False,16.283333
6339565,1,1.61,1.0,100,142,2.0,8.5,0.5,0.5,0.00,...,12.30,2.5,1,31,23,1,31,23,False,9.633333
